<a href="https://colab.research.google.com/github/noriakihanya/getmusic/blob/main/Getmusic_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.13 numpy tensorboard miditoolkit scipy matplotlib tqdm transformers einops
!git clone https://github.com/microsoft/muzic.git

#!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 .
!pip install note-seq
!pip install pyfluidsynth
!pip install midi2audio
!apt-get update -qq && apt-get install -qq libfluidsynth2 build-essential libasound2-dev libjack-dev fluidsynth

!git clone https://github.com/noriakihanya/getmusic.git
import shutil
shutil.copy("/content/getmusic/track_generation.py","/content/muzic/getmusic/")

import gdown
gdown.download('https://drive.google.com/uc?id=1Ub45GMApdXJAGt0_RRbPWwlhIVIHVcT7', 'checkpoint.pth', quiet=False)
gdown.download('https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG', 'MuseScore_General.sf2', quiet=False)

In [2]:
import os
os.makedirs("/content/01_input", exist_ok=True)
os.makedirs("/content/02_output_midi", exist_ok=True)
os.makedirs("/content/03_output_mp3", exist_ok=True)

In [ ]:
filepath = "/content/01_input/basic_pitch_transcription.mid"
midi = mido.MidiFile(filepath)
print(midi.tracks[1][:10])

In [35]:
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

filepath = "/content/01_input/basic_pitch_transcription.mid"
midi = mido.MidiFile(filepath)
# 音符などの情報はmidi.tracks[0]にある
#print(midi.tracks[1][:10])

new_mid = MidiFile()
new_track = MidiTrack()
new_mid.tracks.append(new_track)

new_track.append(MetaMessage('track_name', name='Piano\x00', time=0))

for m in midi.tracks[1][1:]:
  new_track.append(m)

new_mid.save('/content/01_input/new_basic_pitch_transcription.mid')

In [36]:
filepath = "/content/01_input/new_basic_pitch_transcription.mid"
midi = mido.MidiFile(filepath)
# 音符などの情報はmidi.tracks[0]にある
#print(midi.tracks[1][:10])
messages = [m.dict() for m in midi.tracks[0]]
for m in messages[:10]:
  #m["name"] = "Piano"
  print(m)

{'type': 'track_name', 'name': 'Piano\x00', 'time': 0}
{'type': 'program_change', 'time': 0, 'program': 4, 'channel': 0}
{'type': 'note_on', 'time': 1170, 'note': 64, 'velocity': 104, 'channel': 0}
{'type': 'note_on', 'time': 223, 'note': 62, 'velocity': 91, 'channel': 0}
{'type': 'note_off', 'time': 22, 'note': 64, 'velocity': 0, 'channel': 0}
{'type': 'note_off', 'time': 123, 'note': 62, 'velocity': 0, 'channel': 0}
{'type': 'note_on', 'time': 56, 'note': 60, 'velocity': 98, 'channel': 0}
{'type': 'note_off', 'time': 279, 'note': 60, 'velocity': 0, 'channel': 0}
{'type': 'note_on', 'time': 11, 'note': 62, 'velocity': 104, 'channel': 0}
{'type': 'note_on', 'time': 179, 'note': 62, 'velocity': 99, 'channel': 0}


In [38]:
#@title トラックの生成
%cd /content/muzic/getmusic

!python /content/muzic/getmusic/track_generation.py \
--load_path /content/checkpoint.pth \
--file_path /content/01_input \
--output_midi /content/02_output_midi \
--select_condition_tracks p \
--select_content_tracks bcdgps

/content/muzic/getmusic
Global seed set to 0
<class 'getmusic.modeling.models.dfm.DFM'>
<class 'getmusic.modeling.roformer.diffusion_roformer.DiffusionRFM'>
<class 'getmusic.modeling.roformer.roformer_utils.DiffusionRoformerModel'>
Load dictionary: 11879 tokens.
<class 'getmusic.engine.clip_grad_norm.ClipGradNorm'>
Get lr 3e-06 from base lr 3e-06 with none
<class 'torch.optim.adamw.AdamW'>
<class 'getmusic.engine.lr_scheduler.LinearDecayLRWithWarmup'>
{'overall': {'trainable': '86.49M', 'non_trainable': '96.0K', 'total': '86.59M'}, 'rfm': {'trainable': '86.49M', 'non_trainable': '96.0K', 'total': '86.59M'}}
self.device  0
inference_cache: global rank 0: prepare solver done!
Resume from /content/checkpoint.pth
/content/01_input/basic_pitch_transcription.mid
skip?y
/content/01_input/new_basic_pitch_transcription.mid
skip?n
use ema parameters
100% 100/100 [01:29<00:00,  1.11it/s]
sampling, the song has 1035 time units
/content/01_input/worldend.mid
skip?y


In [ ]:
#@title 高度な操作
%cd /content/muzic/getmusic

!python /content/muzic/getmusic/position_generation.py \
--load_path /content/checkpoint.pth \
--file_path /content/01_input

#--file_path /content/01_input
#--file_path /content/muzic/getmusic/example_data/inference

#Input positions you want to condition on: 4,0,
#Input positions you want to empty: 0,0,;6,0,

In [8]:
#@title MIDIの表示
import note_seq

SAMPLE_RATE = 16000
SF2_PATH = '/content/MuseScore_General.sf2'

import glob
for i in glob.glob(f'/content/02_output_midi/*.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    midi_scr = i

est_ns = note_seq.midi_file_to_note_sequence(midi_scr)

note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(est_ns)

/content/02_output_midi/p2bcdgps-世界が終わるまでは_half.mid


In [10]:
#@title MIDIをオーディオフファイルに変換
from pydub import AudioSegment
from midi2audio import FluidSynth
import glob
import math

for i in glob.glob(f'/content/02_output_midi/p2*.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    MTF_FILE = i

fs = FluidSynth(sound_font="/content/MuseScore_General.sf2")
MTF_mp3 = "/content/03_output_mp3/full.mp3"
fs.midi_to_audio(MTF_FILE, MTF_mp3)

audio = AudioSegment.from_file(MTF_mp3)
# 音量(%)
volume_percent = 500

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export(MTF_mp3, format='mp3')

/content/02_output_midi/p2bcdgps-世界が終わるまでは_half.mid


<_io.BufferedRandom name='/content/03_output_mp3/full.mp3'>

In [ ]:
#@title 伴奏を抽出
import mido
from mido import Message, MidiFile, MidiTrack

for i in glob.glob(f'/content/02_output_midi/*.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    midi = mido.MidiFile(i)

gomi = MidiFile()
mid = MidiFile()
track = MidiTrack()

i = 0
for t in midi.tracks:
  i += 1
  gomi.tracks.append(t)
  if i >=3:
    mid.tracks.append(t)

mid.save('/content/02_output_midi/acp_tracks.mid')

/content/output_midi/p2bdgs-よぞらに.mid


In [ ]:
#@title MIDIをオーディオフファイルに変換
from pydub import AudioSegment
from midi2audio import FluidSynth
import glob
import math

for i in glob.glob(f'/content/02_output_midi/acp_tracks.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    MTF_FILE = i

fs = FluidSynth(sound_font="/content/MuseScore_General.sf2")
MTF_mp3 = "/content/03_output_mp3/acp.mp3"
fs.midi_to_audio(MTF_FILE, MTF_mp3)

audio = AudioSegment.from_file(MTF_mp3)
# 音量(%)
volume_percent = 500

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export(MTF_mp3, format='mp3')

/content/output_midi/acp_tracks.mid


<_io.BufferedRandom name='/content/output_mp3/acp.mp3'>